# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 10:06AM,256383,10,Clear-9103197,"ClearSpec, LLC",Released
1,Feb 1 2023 10:06AM,256383,10,Clear-9103200,"ClearSpec, LLC",Released
2,Feb 1 2023 9:43AM,256381,10,Clear-9103179,"ClearSpec, LLC",Released
3,Feb 1 2023 9:42AM,256380,10,PRF-51016,Bio-PRF,Released
4,Feb 1 2023 9:42AM,256380,10,PRF-51022,Bio-PRF,Released
5,Feb 1 2023 9:42AM,256380,10,PRF-51040,Bio-PRF,Released
6,Feb 1 2023 9:42AM,256380,10,PRF-51043,Bio-PRF,Released
7,Feb 1 2023 9:42AM,256380,10,PRF-51046,Bio-PRF,Released
8,Feb 1 2023 9:42AM,256380,10,PRF-51049,Bio-PRF,Released
9,Feb 1 2023 9:42AM,256380,10,PRF-51061,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,256373,Released,1
32,256378,Released,1
33,256380,Released,10
34,256381,Released,1
35,256383,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256373,NaN,NaN,1.0
256378,NaN,NaN,1.0
256380,NaN,NaN,10.0
256381,NaN,NaN,1.0
256383,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256271,0.0,0.0,1.0
256280,0.0,0.0,1.0
256285,0.0,0.0,1.0
256292,0.0,0.0,1.0
256295,2.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256271,0,0,1
256280,0,0,1
256285,0,0,1
256292,0,0,1
256295,2,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256271,0,0,1
1,256280,0,0,1
2,256285,0,0,1
3,256292,0,0,1
4,256295,2,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256271,,,1
1,256280,,,1
2,256285,,,1
3,256292,,,1
4,256295,2,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC"
2,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC"
3,Feb 1 2023 9:42AM,256380,10,Bio-PRF
13,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc."
14,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc."
15,Feb 1 2023 9:26AM,256372,10,Emerginnova
19,Feb 1 2023 9:26AM,256371,10,Emerginnova
21,Feb 1 2023 9:23AM,256370,10,ISDIN Corporation
59,Feb 1 2023 9:23AM,256368,10,ISDIN Corporation
84,Feb 1 2023 9:23AM,256366,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",,,2
1,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC",,,1
2,Feb 1 2023 9:42AM,256380,10,Bio-PRF,,,10
3,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",,,1
4,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc.",,,1
5,Feb 1 2023 9:26AM,256372,10,Emerginnova,,,4
6,Feb 1 2023 9:26AM,256371,10,Emerginnova,,,2
7,Feb 1 2023 9:23AM,256370,10,ISDIN Corporation,,1,37
8,Feb 1 2023 9:23AM,256368,10,ISDIN Corporation,,,25
9,Feb 1 2023 9:23AM,256366,10,ISDIN Corporation,,,47


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",2,,
1,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC",1,,
2,Feb 1 2023 9:42AM,256380,10,Bio-PRF,10,,
3,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",1,,
4,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc.",1,,
5,Feb 1 2023 9:26AM,256372,10,Emerginnova,4,,
6,Feb 1 2023 9:26AM,256371,10,Emerginnova,2,,
7,Feb 1 2023 9:23AM,256370,10,ISDIN Corporation,37,1,
8,Feb 1 2023 9:23AM,256368,10,ISDIN Corporation,25,,
9,Feb 1 2023 9:23AM,256366,10,ISDIN Corporation,47,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",2,,
1,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC",1,,
2,Feb 1 2023 9:42AM,256380,10,Bio-PRF,10,,
3,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",1,,
4,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",2.0,NaN,NaN
1,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC",1.0,NaN,NaN
2,Feb 1 2023 9:42AM,256380,10,Bio-PRF,10.0,NaN,NaN
3,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 10:06AM,256383,10,"ClearSpec, LLC",2.0,0.0,0.0
1,Feb 1 2023 9:43AM,256381,10,"ClearSpec, LLC",1.0,0.0,0.0
2,Feb 1 2023 9:42AM,256380,10,Bio-PRF,10.0,0.0,0.0
3,Feb 1 2023 9:39AM,256378,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Feb 1 2023 9:27AM,256373,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3845250,151.0,3.0,2.0
15,769082,68.0,2.0,0.0
16,769003,2.0,1.0,0.0
19,2050681,30.0,14.0,8.0
21,256316,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3845250,151.0,3.0,2.0
1,15,769082,68.0,2.0,0.0
2,16,769003,2.0,1.0,0.0
3,19,2050681,30.0,14.0,8.0
4,21,256316,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,151.0,3.0,2.0
1,15,68.0,2.0,0.0
2,16,2.0,1.0,0.0
3,19,30.0,14.0,8.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,151.0
1,15,Released,68.0
2,16,Released,2.0
3,19,Released,30.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,2.0,0.0,0.0,8.0,0.0
Executing,3.0,2.0,1.0,14.0,1.0
Released,151.0,68.0,2.0,30.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,2.0,0.0,0.0,8.0,0.0
1,Executing,3.0,2.0,1.0,14.0,1.0
2,Released,151.0,68.0,2.0,30.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,2.0,0.0,0.0,8.0,0.0
1,Executing,3.0,2.0,1.0,14.0,1.0
2,Released,151.0,68.0,2.0,30.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()